# **Plan du projet**

### **1.** Importation des données et constitution de la DataFrame
1.   Fonctions permettant l'importation des données
2.   Base comparateur des territoires en 2017 - INSEE
3.   Base salaire net horaire moyen en 2017 - INSEE
4.   Base scolarisation en 2017 - INSEE
5.   Consolidation des DataFrames
6.   Base établissements d'enseignement du premier et second degrés




In [1]:
!pip install geopandas

import pandas as pd
import numpy
import geopandas as gpd

import bs4
import re

import matplotlib.pyplot as plt
%matplotlib inline

from io import BytesIO
from zipfile import ZipFile
from urllib.request import urlopen

from functools import reduce

     |████████████████████████████████| 972kB 10.8MB/s 
     |████████████████████████████████| 6.5MB 47.4MB/s 
     |████████████████████████████████| 14.8MB 296kB/s 


# **Importation des données et constitution de la DataFrame**


## Fonctions permettant l'importation des données


La première fonction (**csv_zip_to_df**) permet de convertir un **fichier CSV en format ZIP** grâce à son URL et au nom du fichier en un **dataframe**.

La deuxième fonction (**taux**) permet de **créer taux** correspondant au rapport de 2 autres variables initialement dans la base de données. La variable créée est **arrondie au centième**.

In [2]:
def csv_zip_to_df (url_file,file_name):
  url = urlopen(url_file)
  zipfile = ZipFile(BytesIO(url.read()))
  dataframe = pd.read_csv(zipfile.open(file_name), header = 0,sep=";")
  return dataframe

def taux(new_var,nom_var,denom_var,bdd):
  bdd[new_var] = bdd[nom_var] / bdd[denom_var]
  bdd[new_var] = bdd[new_var].round(decimals=3)
  return bdd[new_var]

## Base comparateur des territoires en 2017 - INSEE
Cette base, issue de l'INSEE (https://www.insee.fr/fr/statistiques/2521169) renvoie des **indicateurs économiques** à l'échelle communale. 

Pour notre étude, nous n'avons pas besoin de l'intégralité des variables présentes sur cette base, nous allons en sélectionner quelques unes : 

- Population en 2017 (**P17_POP**);
- Médiane du niveau vie en 2017 (**MED17**);

Et en créer certaines :
- Densité en 2017 (**DEN17**);
- Taux d'activité en 2017 (**T17_ACT1564**);
- Taux de chômage en 2017 (**T17_CHOM1564**).

In [ ]:
bdd_ind_eco = csv_zip_to_df("https://www.insee.fr/fr/statistiques/fichier/2521169/base-comparateur-2017_CSV.zip","base_cc_comparateur.CSV")

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2822: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [ ]:
#Création du taux de chômage et d'activité, et de la densité
taux('T17_ACT1564','P17_ACT1564','P17_POP1564',bdd_ind_eco)
taux('T17_CHOM1564','P17_CHOM1564','P17_POP1564',bdd_ind_eco)
taux('DEN17','P17_POP','SUPERF',bdd_ind_eco)

#Sélection de variables et arrondis
bdd_ind_eco = bdd_ind_eco[['CODGEO','P17_POP','DEN17','MED17','P17_EMPLT','T17_ACT1564','T17_CHOM1564']]

bdd_ind_eco[['T17_ACT1564','T17_CHOM1564']] = bdd_ind_eco[['T17_ACT1564','T17_CHOM1564']].round(decimals=3)
bdd_ind_eco[['DEN17']] = bdd_ind_eco[['DEN17']].round(decimals=0)

bdd_ind_eco

,CODGEO,P17_POP,DEN17,MED17,P17_EMPLT,T17_ACT1564,T17_CHOM1564
0,01001,776.0,49.0,23310.0,98.828563,0.812,0.071
1,01002,248.0,27.0,24290.0,16.169223,0.854,0.069
2,01004,14035.0,571.0,19860.0,7685.583424,0.762,0.113
3,01005,1689.0,106.0,23370.0,293.563139,0.807,0.063
4,01006,111.0,19.0,23970.0,6.054545,0.806,0.111
...,...,...,...,...,...,...,...
34994,75116,166361.0,10163.0,38900.0,118108.644980,0.732,0.073
34995,75117,167288.0,29504.0,30800.0,106753.699959,0.799,0.090
34996,75118,195233.0,32485.0,22330.0,88116.745765,0.801,0.102
34997,75119,187015.0,27543.0,20320.0,89180.620928,0.767,0.126


## Base salaire net horaire moyen en 2017 - INSEE
Cette base, issue de l'INSEE (https://www.insee.fr/fr/statistiques/2021266) renvoie des indicateurs sur le **salaire horaire net moyen par tranches d'âge, par sexes et par catégories socio-professionnelles**, à l'échelle communale. 





Nous allons garder les variables suivantes : 

- Salaire net horaire moyen en 2017 (€) (**SHOR17**);
- Salaire net horaire moyen F en 2017 (€) (**SHOR17_F**);
- Salaire net horaire moyen H en 2017 (€) (**SHOR17_H**);
- Salaire net hor. Moy. Cadres sup. en 2017 (€) (**SHOR17_CS**);
- Salaire net hor. moy. prof inter. en 2017 (€) (**SHOR17_PI**);
- Salaire net hor. moy. employés en 2017 (€) (**SHOR17_E**);
- Salaire net hor. moy. ouvriers en 2017 (€) (**SHOR17_O**);
- Salaire net horaire moyen 18 à 25 ans en 2017 (€) (**SHOR17_1825**);
- Salaire net horaire moyen 26 à 50 ans en 2017 (€) (**SHOR17_2650**);
- Salaire net horaire moyen plus de 50 ans en 2017 (€) (**SHOR17_50P**).

In [ ]:
bdd_sal = csv_zip_to_df ("https://www.insee.fr/fr/statistiques/fichier/2021266/base-cc-dads-2017_csv.zip","cc_dads_2017_COM.csv")

In [ ]:
#Sélection de variables et arrondis
bdd_sal = bdd_sal [['CODGEO','SNHM17','SNHMF17','SNHMH17','SNHMC17','SNHMP17','SNHME17','SNHMO17','SNHM1817','SNHM2617','SNHM5017']].round(decimals=2)
bdd_sal = bdd_sal.rename(columns={'SNHM17':'SHOR17','SNHMF17':'SHOR17_F','SNHMH17':'SHOR17_H','SNHMC17':'SHOR17_CS','SNHMP17':'SHOR17_PI','SNHME17':'SHOR17_E','SNHMO17':'SHOR17_O','SNHM1817':'SHOR17_1825','SNHM2617':'SHOR17_2650','SNHM5017':'SHOR17_50P'})

bdd_sal

,CODGEO,SHOR17,SHOR17_F,SHOR17_H,SHOR17_CS,SHOR17_PI,SHOR17_E,SHOR17_O,SHOR17_1825,SHOR17_2650,SHOR17_50P
0,01004,14.18,12.12,15.40,24.74,15.97,10.46,11.10,10.17,14.25,16.61
1,01007,13.67,12.53,14.59,21.95,14.58,10.72,11.16,9.90,13.84,14.65
2,01014,15.30,11.84,17.62,39.89,15.71,11.15,10.94,10.39,15.74,15.99
3,01024,13.15,11.71,14.16,22.10,14.77,10.92,11.12,9.73,13.34,14.18
4,01025,13.37,12.25,14.09,23.12,14.48,10.80,11.27,9.45,13.27,15.17
...,...,...,...,...,...,...,...,...,...,...,...
5358,75116,38.32,26.07,49.34,51.93,32.35,12.60,11.57,20.32,34.10,51.91
5359,75117,26.85,22.67,30.76,36.37,17.54,12.10,11.21,12.03,26.14,34.93
5360,75118,18.60,17.92,19.15,26.50,15.67,11.34,10.62,11.06,18.88,20.86
5361,75119,16.44,15.89,16.86,24.78,15.25,11.08,10.46,10.40,16.70,17.95


## Base scolarisation en 2017 - INSEE
Cette base, issue de l'INSEE (https://www.insee.fr/fr/statistiques/4516086?sommaire=4516089) renvoie des indicateurs sur la **scolarisation par catégories d'âge et la part des diplômés dans la population active** *(personnes non scolarisées de 15 ans ou plus)* **selon le diplôme**, à l'échelle communale. 


Nous allons sélectionner certaines variables afin de créer les suivantes :
- Taux de scolarisation entre 2 et 5 ans (**T17_0205**);
- Taux de scolarisation entre 6 et 10 ans (**T17_0610**);
- Taux de scolarisation entre 11 et 14 ans (**T17_1114**);
- Taux de scolarisation entre 15 et 17 ans (**T17_1517**);
- Taux de scolarisation entre 18 et 24 ans (**T17_1824**);
- Taux de scolarisation entre 25 et 29 ans (**T17_2529**);


- Part de la population active n'ayant pas ou peu de diplôme (**T17_NDIP**);
- Part de la population active dont le diplôme le plus élevé est le BEPC ou le brevet (**T17_BEPC**);
- Part de la population active dont le diplôme le plus élevé est un CAP ou BEP (**T17_CAPBEP**);
- Part de la population active dont le diplôme le plus élevé est le BAC (**T17_BAC**);
- Part des diplômés d'un BAC+2 dans la population active (**T17_SUP2**);
- Part des diplômés d'un BAC+3 ou BAC+4 dans la population active (**T17_SUP34**);
- Part des diplômés d'un BAC+5 ou plus dans la population active (**T17_SUP5**).


In [ ]:
bdd_scolar = csv_zip_to_df("https://www.insee.fr/fr/statistiques/fichier/4516086/base-ccc-diplomes-formation-2017.zip","base-cc-diplomes-formation-2017.CSV")

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2822: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [ ]:
#Création des taux de scolarisation
taux('T17_0205','P17_SCOL0205','P17_POP0205',bdd_scolar)
taux('T17_0610','P17_SCOL0610','P17_POP0610',bdd_scolar)
taux('T17_1114','P17_SCOL1114','P17_POP1114',bdd_scolar)
taux('T17_1517','P17_SCOL1517','P17_POP1517',bdd_scolar)
taux('T17_1824','P17_SCOL1824','P17_POP1824',bdd_scolar)
taux('T17_2529','P17_SCOL2529','P17_POP2529',bdd_scolar)

#Part de diplômés
taux('T17_NDIP','P17_NSCOL15P_DIPLMIN','P17_NSCOL15P',bdd_scolar)
taux('T17_BEPC','P17_NSCOL15P_BEPC','P17_NSCOL15P',bdd_scolar)
taux('T17_CAPBEP','P17_NSCOL15P_CAPBEP','P17_NSCOL15P',bdd_scolar)
taux('T17_BAC','P17_NSCOL15P_BAC','P17_NSCOL15P',bdd_scolar)
taux('T17_SUP2','P17_NSCOL15P_SUP2','P17_NSCOL15P',bdd_scolar)
taux('T17_SUP34','P17_NSCOL15P_SUP34','P17_NSCOL15P',bdd_scolar)
taux('T17_SUP5','P17_NSCOL15P_SUP5','P17_NSCOL15P',bdd_scolar)

0        0.056
1        0.091
2        0.063
3        0.048
4        0.032
         ...  
34991    0.043
34992    0.017
34993    0.034
34994    0.024
34995    0.011
Name: T17_SUP5, Length: 34996, dtype: float64

In [ ]:
bdd_scolar['CODGEO']= bdd_scolar['CODGEO'].astype(str).str.zfill(5)
bdd_scolar['DEP']=bdd_scolar['CODGEO'].str[:2]

#Sélection de variables
bdd_scolar = bdd_scolar[['CODGEO','DEP','T17_0205','T17_0610','T17_1114','T17_1517','T17_1824','T17_2529','T17_NDIP','T17_BEPC','T17_CAPBEP','T17_BAC','T17_SUP2','T17_SUP34','T17_SUP5']]
bdd_scolar

## Base établissements d'enseignement du premier et second degrés

In [ ]:
import csv
import io
import urllib.request

#url = "https://data.education.gouv.fr/explore/dataset/fr-en-adresse-et-geolocalisation-etablissements-premier-et-second-degre/download/?format=csv&timezone=Europe/Berlin&lang=fr&use_labels_for_header=true&csv_separator=%3B/fr-en-adresse-et-geolocalisation-etablissements-premier-et-second-degre.csv"
url = "https://raw.github.com/datasets/gdp/master/data/gdp.csv"
webpage = urllib.request.urlopen(url)
datareader = csv.reader(io.TextIOWrapper(webpage))
data = list(datareader)

#for row in datareader:
   # print(row)

In [9]:
#Ca marche mais c'est hyper lent et ça importe que les 1000 premières observations sur 65000
import requests
data_json = requests.get("https://data.education.gouv.fr/explore/dataset/fr-en-adresse-et-geolocalisation-etablissements-premier-et-second-degre/download/?format=json").json()
data_json

[{'datasetid': 'fr-en-adresse-et-geolocalisation-etablissements-premier-et-second-degre',
  'fields': {'appariement': 'Correcte',
   'appellation_officielle': 'Ecole maternelle Les Quatre Vents',
   'code_academie': '23',
   'code_commune': '83125',
   'code_departement': '83',
   'code_ministere': '06',
   'code_postal_uai': '83470',
   'code_region': '93',
   'coordonnee_x': 933456.6,
   'coordonnee_y': 6270783.5,
   'date_ouverture': '1997-08-20',
   'denomination_principale': 'ECOLE MATERNELLE PUBLIQUE',
   'epsg': 'EPSG:2154',
   'etat_etablissement': 1,
   'etat_etablissement_libe': 'OUVERT',
   'latitude': 43.49773683037158,
   'libelle_academie': 'Nice',
   'libelle_commune': "Seillons-Source-d'Argens",
   'libelle_departement': 'Var',
   'libelle_ministere': "MINISTERE DE L'EDUCATION NATIONALE",
   'libelle_region': "Provence-Alpes-Côte d'Azur",
   'lieu_dit_uai': 'LES AIRES',
   'localisation': 'Rue',
   'localite_acheminement_uai': 'SEILLONS SOURCE D ARGENS',
   'longitude':

In [ ]:
data_json[0]

In [17]:
# JSON file 
f = open ('data.json', "r") 
  
# Reading from file 
data = json.loads(f.read()) 
  
# Iterating through the json 
# list 
for i in data['emp_details']: 
    print(i) 

SyntaxError: ignored

## Consolidation des DataFrames

In [ ]:
#Création d'une liste contenant les dataframes à consolider
liste_df = [bdd_scolar,bdd_ind_eco, bdd_sal]

df_final = reduce(lambda left,right: pd.merge(left, right, on='CODGEO',left_index=True, right_index=True, how='outer'), liste_df)

df_final.head()

,CODGEO,DEP,T17_0205,T17_0610,T17_1114,T17_1517,T17_1824,T17_2529,T17_NDIP,T17_BEPC,T17_CAPBEP,T17_BAC,T17_SUP2,T17_SUP34,T17_SUP5,P17_POP,DEN17,MED17,P17_EMPLT,T17_ACT1564,T17_CHOM1564,SHOR17,SHOR17_F,SHOR17_H,SHOR17_CS,SHOR17_PI,SHOR17_E,SHOR17_O,SHOR17_1825,SHOR17_2650,SHOR17_50P
0,01001,01,0.679,0.965,1.000,0.970,0.554,0.061,0.192,0.047,0.335,0.170,0.129,0.071,0.056,776.0,49.0,23310.0,98.828563,0.812,0.071,14.18,12.12,15.40,24.74,15.97,10.46,11.10,10.17,14.25,16.61
1,01002,01,0.687,1.000,1.000,1.000,0.375,0.077,0.126,0.040,0.269,0.217,0.143,0.114,0.091,248.0,27.0,24290.0,16.169223,0.854,0.069,13.67,12.53,14.59,21.95,14.58,10.72,11.16,9.90,13.84,14.65
2,01004,01,0.719,0.989,0.989,0.953,0.387,0.025,0.217,0.058,0.276,0.191,0.122,0.073,0.063,14035.0,571.0,19860.0,7685.583424,0.762,0.113,15.30,11.84,17.62,39.89,15.71,11.15,10.94,10.39,15.74,15.99
3,01005,01,0.723,0.974,0.977,0.948,0.368,0.000,0.238,0.045,0.301,0.179,0.120,0.069,0.048,1689.0,106.0,23370.0,293.563139,0.807,0.063,13.15,11.71,14.16,22.10,14.77,10.92,11.12,9.73,13.34,14.18
4,01006,01,1.000,1.000,1.000,1.000,0.000,0.000,0.179,0.042,0.326,0.316,0.053,0.053,0.032,111.0,19.0,23970.0,6.054545,0.806,0.111,13.37,12.25,14.09,23.12,14.48,10.80,11.27,9.45,13.27,15.17
